In [2]:
import pandas as pd
import numpy as np

In [9]:
import pickle
import json
import seaborn as sns
import matplotlib.transforms as mtransforms
from bread import vis
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bread.algo import tracking
from bread.data import *
from sklearn.metrics import confusion_matrix
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import sys
import os.path
import datetime, json
from bread.algo.tracking import AssignmentDataset, GNNTracker, AssignmentClassifier, GraphLoader, accuracy_assignment, f1_assignment
from bread.data import *
from bread.algo.tracking import AssignmentDataset, GNNTracker, AssignmentClassifier, GraphLoader, accuracy_assignment, f1_assignment

from glob import glob
from pathlib import Path
import datetime, json, argparse
import torch, os
from pprint import pprint
from sklearn.metrics import confusion_matrix


from skorch.dataset import ValidSplit
from skorch.callbacks import LRScheduler, Checkpoint, ProgressBar, EpochScoring, WandbLogger, EarlyStopping

from importlib import import_module

import tqdm
import numpy as np, scipy.optimize, scipy.sparse



In [4]:
res_hungarian_path = '/mlodata1/hokarami/fari/bread/best_models/f10_f10/2023-10-29 20:35:20/results/hungarian_2023-10-30 15:13:54/result_test_set_6.csv'

In [21]:
res_hungarian = pd.read_csv(res_hungarian_path)

In [22]:
res_hungarian = res_hungarian.loc[res_hungarian['f1']<1.0]
res_hungarian = res_hungarian.loc[res_hungarian['timediff']<15.0]
# res_hungarian = res_hungarian.loc[res_hungarian['fp']+res_hungarian['fn']>=4]
res_hungarian

,t1,t2,colony,num_cells1,num_cells2,method,tp,fp,tn,fn,acc,f1,timediff
125,31,33,colony000,8,12,gcn,7,1,87,1,0.979167,0.875000,10
189,47,49,colony000,16,19,gcn,15,1,287,1,0.993421,0.937500,10
193,48,50,colony000,18,19,gcn,17,1,323,1,0.994152,0.944444,10
296,74,75,colony000,44,44,gcn,43,1,1892,0,0.999483,0.988506,5
297,74,76,colony000,44,44,gcn,43,1,1892,0,0.999483,0.988506,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5059,2,3,test_set_3,101,102,gcn,100,1,10200,1,0.999806,0.990099,5
5060,2,4,test_set_3,101,102,gcn,100,1,10200,1,0.999806,0.990099,10
5084,8,10,test_set_3,108,118,gcn,107,1,12635,1,0.999843,0.990741,10
5087,9,10,test_set_3,114,118,gcn,113,1,13337,1,0.999851,0.991228,5


In [23]:
model_folder = '/mlodata1/hokarami/fari/bread/best_models/f10_f10/2023-10-29 20:35:20'
model_dir = Path(model_folder)
algo = 'hungarian'
resultdir = Path(f'{model_dir}/results/{algo}_{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

with open(model_dir / 'hyperparams.json') as file:
    hparams = json.load(file)

net = AssignmentClassifier(
    GNNTracker,
    module__num_node_attr=hparams['num_node_attr'],
    module__num_edge_attr=hparams['num_edge_attr'],
    module__dropout_rate=hparams['dropout_rate'],
    module__encoder_hidden_channels=hparams['encoder_hidden_channels'],
    module__encoder_num_layers=hparams['encoder_num_layers'],
    module__conv_hidden_channels=hparams['conv_hidden_channels'],
    module__conv_num_layers=hparams['conv_num_layers'],
    module__num_classes=1,
    iterator_train=GraphLoader,
    iterator_valid=GraphLoader,
    criterion=torch.nn.BCEWithLogitsLoss,
).initialize()
net.load_params(model_dir / 'params.pt')
# disable training
net.module_.train(False) 

GNNTracker(
  (node_encoder): MLP(60, 120, 120, 120, 120)
  (edge_encoder): MLP(10, 120, 120, 120, 120)
  (layers): ModuleList(
    (0-2): 3 x DeepGCNLayer(block=res+)
  )
  (out): MLP(120, 60, 30, 1)
)

In [15]:
# /mlodata1/hokarami/fari/bread/data/generated/test_tracking/ass_graphs_original/colony003_segmentation__assgraph__dt_004__001_to_005.pt
assignment_graph_dir = Path("/mlodata1/hokarami/fari/bread/data/generated/test_tracking/ass_graphs_fourier_10_and_f10_locality_False")

# iterate over rows in res_hungarian
all_percentage = []
for index , row in res_hungarian.iterrows():
    colony_name = row["colony"]
    time_diff = int(row['timediff']/5)
    first_frame = int(row['t1'])
    next_frame =  int(row['t2'])
    ass_graph_path = assignment_graph_dir/f'{colony_name}_segmentation__assgraph__dt_{time_diff:03d}__{first_frame:03d}_to_{next_frame:03d}.pt'
    graph = torch.load(ass_graph_path)
    
    scores = net.predict_raw(graph)
    row_ind, col_ind = scipy.optimize.linear_sum_assignment(scores, maximize=True)  # maximizing the total assignment score

    y = graph.y.squeeze().cpu().numpy()
    y = y.reshape(scores.shape)
    
    
    # 'row_ind' and 'col_ind' are the row and column indices of the optimal assignments.
    # Create the binary assignment matrix
    binary_assignment_matrix = np.zeros_like(scores, dtype=int)
    binary_assignment_matrix[row_ind, col_ind] = 1

    # Calculate the percentage of assignment scores that are higher for each assigned cell
    percentage_higher_scores = []
    threshold = 10  # Adjust the threshold as needed
    for i, j in zip(row_ind, col_ind):
        if (y[i,j]==0 and binary_assignment_matrix[i, j] == 1):
            count_higher_scores = np.sum(scores[:, j] > scores[i, j])
            percentage = (count_higher_scores / (scores.shape[0] - 1)) * 100  # excluding the current cell
            percentage_higher_scores.append(percentage)
            
    print(colony_name,": ",np.mean(percentage_higher_scores))
    all_percentage.extend(percentage_higher_scores)
print("mean_all", np.mean(all_percentage))
print("std_all", np.std(all_percentage))
print("max_all", np.max(all_percentage))
print("min_all", np.min(all_percentage))



colony000 :  10.119047619047619
colony000 :  1.9230769230769231
colony006 :  10.38961038961039
colony006 :  0.5263157894736842
colony002 :  1.0309278350515463
colony002 :  24.31192660550459
colony002 :  24.545454545454543
colony002 :  24.545454545454543
colony004 :  13.580246913580245
colony004 :  12.12121212121212
colony004 :  23.636363636363637
colony004 :  3.481012658227848
colony004 :  4.065040650406504
colony004 :  1.8292682926829267
colony004 :  12.4031007751938
colony004 :  15.517241379310343
colony004 :  0.7662835249042145
colony004 :  6.443298969072165
colony005 :  32.857142857142854
colony005 :  22.857142857142858
colony005 :  22.857142857142858
colony008 :  8.994708994708995
colony008 :  8.717948717948719
colony008 :  4.615384615384616
mean_all 11.05256149469232
std_all 16.087058120686812
max_all 62.857142857142854
min_all 0.0


In [18]:
res_hungarian_all = pd.read_csv(res_hungarian_path)

In [20]:
# /mlodata1/hokarami/fari/bread/data/generated/test_tracking/ass_graphs_original/colony003_segmentation__assgraph__dt_004__001_to_005.pt
assignment_graph_dir = Path("/mlodata1/hokarami/fari/bread/data/generated/test_tracking/ass_graphs_fourier_10_and_f10_locality_False")

# iterate over rows in res_hungarian
all_percentage = []
for index , row in res_hungarian_all.iterrows():
    colony_name = row["colony"]
    time_diff = int(row['timediff']/5)
    first_frame = int(row['t1'])
    next_frame =  int(row['t2'])
    ass_graph_path = assignment_graph_dir/f'{colony_name}_segmentation__assgraph__dt_{time_diff:03d}__{first_frame:03d}_to_{next_frame:03d}.pt'
    graph = torch.load(ass_graph_path)
    
    scores = net.predict_raw(graph)
    row_ind, col_ind = scipy.optimize.linear_sum_assignment(scores, maximize=True)  # maximizing the total assignment score

    y = graph.y.squeeze().cpu().numpy()
    y = y.reshape(scores.shape)
    
    
    # 'row_ind' and 'col_ind' are the row and column indices of the optimal assignments.
    # Create the binary assignment matrix
    binary_assignment_matrix = np.zeros_like(scores, dtype=int)
    binary_assignment_matrix[row_ind, col_ind] = 1

    # Calculate the percentage of assignment scores that are higher for each assigned cell
    percentage_higher_scores = []
    threshold = 10  # Adjust the threshold as needed
    for i, j in zip(row_ind, col_ind):
        if (y[i,j]==0 and binary_assignment_matrix[i, j] == 1):
            count_higher_scores = np.sum(scores[:, j] > scores[i, j])
            percentage = (count_higher_scores / (scores.shape[0] - 1)) * 100  # excluding the current cell
            percentage_higher_scores.append(percentage)
    print(colony_name,": ",np.mean(percentage_higher_scores))
    all_percentage.extend(percentage_higher_scores)
print("mean_all", np.mean(all_percentage))
print("std_all", np.std(all_percentage))
print("max_all", np.max(all_percentage))
print("min_all", np.min(all_percentage))



colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  nan
colony000 :  n